# CMS to PCORNet Terminology Mapping

The naive code building approach in `cms_pd.CMSRIFUpload.pivot_valtype`  also results in demographic codes such as `BENE_SEX_IDENT_CD:2` for @@ and `BENE_RACE_CD:1` for @@ where [PCORNet CDM](http://www.pcornet.org/pcornet-common-data-model/) uses @@...


[Data Dictionaries - Chronic Conditions Data Warehouse](https://www.ccwdata.org/web/guest/data-dictionaries)


In [ ]:
from cms_code_table import Cache


class CMSDataDictionaries(Cache):
    [mbsf_abcd] = [
        ('Master Beneficiary Summary - Base (A/B/C/D) Codebook',
         'https://www.ccwdata.org/documents/10280/19022436/codebook-mbsf-abcd.pdf',
         '2f7fce7c849e011d125a8487833e6cdd4ca7ced7')
    ]


class PCORNetCDM(Cache):
    [v3dot1] = [
        ('PCORnet Common Data Model v3.1 Parseable Spreadsheet Format',
         'http://www.pcornet.org/wp-content/uploads/2017/01/2017-01-06-PCORnet-Common-Data-Model-v3dot1-parseable.xlsx',
         'f085b975ec5e59bef3bf505aaa3107e3f4e12e4c')
    ]


def _mk_caches(cache_dir_name='cache'):
    from pathlib import Path
    from urllib.request import build_opener

    web_ua = build_opener()
    cache_dir = Path(cache_dir_name)
    cms = CMSDataDictionaries(cache_dir, web_ua)
    cdm = PCORNetCDM(cache_dir, web_ua)
    return cms, cdm

cms_cache, cdm_cache = _mk_caches()
[mbsf_abcd_codebook, pcornet_cdm_v3dot1] = [
    cms_cache[cms_cache.mbsf_abcd], cdm_cache[cdm_cache.v3dot1]]

[mbsf_abcd_codebook, pcornet_cdm_v3dot1]

Then convert to text using [poppler](http://poppler.freedesktop.org/) tools...

    Package: poppler-utils
    Original-Maintainer: Loic Minier <lool@dooz.org>
    Architecture: amd64
    Version: 0.41.0-0ubuntu1
    Size: 130156
    SHA1: 03e35d9c79455b01cffcbc635219bdb665067740
    SHA256: 995e6af26b24f539df466df997dce3f391b9698097d6c2e691857eb88ce746b8
    Description-en: PDF utilities (based on Poppler)
    Homepage: http://poppler.freedesktop.org/

In [ ]:
!pdftotext -layout cache/codebook-mbsf-abcd.pdf

In [ ]:
import pandas as pd
dict(pandas=pd.__version__)

In [ ]:
mbsf_abcd_codebook.with_suffix('.txt')

In [ ]:
codebook_text = pd.DataFrame({
    'line': [line.strip('\f\n') for line in
             mbsf_abcd_codebook.with_suffix('.txt').open().readlines()]})
codebook_text[:3]

In [ ]:
toc_dots = codebook_text.line.str.match(r'.*\.\.\.')
codebook_text[toc_dots].iloc[-2:]

In [ ]:
codebook_text.iloc[232]

In [ ]:
is_footer = pd.Series(False, index=codebook_text.index)
for footer_pattern in [
    r'^\s+\^ Back to TOC \^',
    r'^CMS Chronic Conditions Data Warehouse \(CCW\)',
    r'^Master Beneficiary Summary File \(MBSF\) with',
    r'^May 2017 – Version 1.0  \s*  Page \d+ of \d+$']:
    is_footer = is_footer | codebook_text.line.str.match(footer_pattern) 

codebook_text[is_footer].head(10)

In [ ]:
codebook_text['is_body'] = ~is_footer & (codebook_text.index > toc_dots[toc_dots].index.max())

In [ ]:
codebook_text['variable'] = codebook_text.line.str.extract(r'^   \s*([A-Z_0-9]+)$', expand=False)

def extract_lhs_rhs(line):
    """
    e.g. SHORT NAME: B_MO_CNT
    also rhs with empty lhs from comment, description
    """
    return line.str.extract(
         r'^(?:(?P<lhs>[A-Z ]+):\s+|            \s*)(?P<rhs>\S.*)', expand=True)

def extract_valuesets(line):
    return line.str.extract(
        r' (?P<valueset_item>\S+) = (?P<valueset_item_descriptor>.*)', expand=True)


codebook = pd.concat([codebook_text,
                      extract_lhs_rhs(codebook_text.line),
                      extract_valuesets(codebook_text.line)], axis=1)

codebook.loc[~codebook.variable.isnull(), 'lhs'] = 'variable'
codebook.lhs = codebook.lhs.fillna(method='pad')
codebook.variable = codebook.variable.fillna(method='pad')



def para_agg(df, text_cols, index, columns, values,
             sep='\n'):
    return (
        df[df.lhs.isin(text_cols)]
        .groupby([index, columns])[values]
        .apply(lambda values: sep.join(values))
        .reset_index()
        .pivot(index=index, columns=columns, values=values))

# &           ~codebook.lhs.isin(['variable', 'VALUES', 'CODE VALUES']                                    
#para_agg(codebook, ['COMMENT', 'DESCRIPTION'], 'variable', 'lhs', 'rhs')
# VALUES: 0-12 @@

codebook = codebook[codebook.is_body & (codebook.line > '')]
#codebook = codebook.pivot(index='variable', values='rhs', columns='lhs')
codebook_values = codebook[~codebook.valueset_item.isnull()][['variable', 'valueset_item', 'valueset_item_descriptor']]
#codebook_values.head(30)

def find_pivot_dups(df, index, columns, values):
    x = df.groupby([index, columns])[[values]].count()
    return x[x[values] > 1]

codebook_variables = pd.concat(
    [codebook[~codebook.lhs.isin(['variable', 'COMMENT', 'DESCRIPTION', 'VALUES', 'CODE VALUES'])
             ].pivot(index='variable', columns='lhs', values='rhs'),
     para_agg(codebook, ['COMMENT', 'DESCRIPTION'], 'variable', 'lhs', 'rhs')], axis=1)
codebook_variables.head()

In [ ]:
codebook_values[codebook_values.variable == 'SEX']

## More @@@

In [ ]:
pcornet_cdm = pd.read_excel(str(pcornet_cdm_v3dot1), sheetname=None)

In [ ]:
pcornet_cdm.keys()

In [ ]:
pcornet_value = pcornet_cdm['VALUESETS']
pcornet_value[pcornet_value.FIELD_NAME == 'SEX']